<a href="https://colab.research.google.com/github/Cumbercookie/CNN-ResNet-and-VGG/blob/main/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras import models

In [2]:
# Load data
labels_df = pd.read_csv('/content/chineseMNIST.csv')

# Data preprocessing
X_data = labels_df.drop(columns=['label', 'character']).values.reshape(-1, 64, 64, 1).astype('float32') / 255.0
y_data = labels_df['label'].values

# Filter out invalid labels
valid_labels = set(range(15))  # Assuming there are 15 classes
valid_indices = labels_df['label'].isin(valid_labels)
X_data_filtered = X_data[valid_indices]
y_data_filtered = y_data[valid_indices]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_data_filtered, y_data_filtered, test_size=0.2, random_state=42)

# Define input shape and number of classes
input_shape = X_train.shape[1:]
num_classes = len(valid_labels)

In [3]:
# Create CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [7]:
from tensorflow.keras.layers import Input, Concatenate, GlobalAveragePooling2D

def create_resnet_model(input_shape, num_classes):
    # Define input with three channels
    input_tensor = Input(shape=input_shape)
    input_rgb = Concatenate()([input_tensor, input_tensor, input_tensor])  # Convert grayscale to RGB

    # Load ResNet50 model with weights
    resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_rgb)

    # Freeze layers in ResNet model
    for layer in resnet_base.layers:
        layer.trainable = False

    # Add custom layers on top of ResNet base
    x = GlobalAveragePooling2D()(resnet_base.output)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = models.Model(inputs=input_tensor, outputs=output)

    return model

In [27]:
# Print input shape for verification
print("Input shape:", input_shape)

Input shape: (64, 64, 3)


In [31]:
# Convert grayscale images to RGB
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_val_rgb = np.repeat(X_val, 3, axis=-1)

# Adjust input shape for VGG model
input_shape = X_train_rgb.shape[1:]

# Create VGG model
def create_vgg_model(input_shape, num_classes):
    vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        vgg_base,
        Flatten(),
        Dense(256, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [32]:
# Train models
def train_model(model, X_train, y_train, X_val, y_val, batch_size=32, epochs=5):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))
    return history


In [12]:
# Train CNN model
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_history = train_model(cnn_model, X_train, y_train, X_val, y_val)


Epoch 1/5
275/275 [==============================] - 51s 182ms/step - loss: 0.7606 - accuracy: 0.7460 - val_loss: 0.2050 - val_accuracy: 0.9332
Epoch 2/5
275/275 [==============================] - 48s 176ms/step - loss: 0.1527 - accuracy: 0.9515 - val_loss: 0.1864 - val_accuracy: 0.9382
Epoch 3/5
275/275 [==============================] - 52s 190ms/step - loss: 0.0973 - accuracy: 0.9683 - val_loss: 0.1179 - val_accuracy: 0.9618
Epoch 4/5
275/275 [==============================] - 51s 185ms/step - loss: 0.0567 - accuracy: 0.9817 - val_loss: 0.1294 - val_accuracy: 0.9591
Epoch 5/5
275/275 [==============================] - 51s 187ms/step - loss: 0.0428 - accuracy: 0.9860 - val_loss: 0.0497 - val_accuracy: 0.9832


In [15]:
# Train ResNet model
resnet_model = create_resnet_model(input_shape, num_classes)
resnet_history = train_model(resnet_model, X_train, y_train, X_val, y_val)


Epoch 1/5
275/275 [==============================] - 230s 825ms/step - loss: 2.1979 - accuracy: 0.2361 - val_loss: 1.9298 - val_accuracy: 0.3782
Epoch 2/5
275/275 [==============================] - 165s 599ms/step - loss: 1.7898 - accuracy: 0.4189 - val_loss: 1.6543 - val_accuracy: 0.4455
Epoch 3/5
275/275 [==============================] - 175s 636ms/step - loss: 1.5748 - accuracy: 0.4798 - val_loss: 1.4431 - val_accuracy: 0.5364
Epoch 4/5
275/275 [==============================] - 172s 627ms/step - loss: 1.4310 - accuracy: 0.5283 - val_loss: 1.3622 - val_accuracy: 0.5532
Epoch 5/5
275/275 [==============================] - 173s 631ms/step - loss: 1.3155 - accuracy: 0.5748 - val_loss: 1.2486 - val_accuracy: 0.6041


In [34]:
# Train VGG model
vgg_model = create_vgg_model(input_shape, num_classes)
vgg_history = train_model(vgg_model, X_train_rgb, y_train, X_val_rgb, y_val)

Epoch 1/5
275/275 [==============================] - 1787s 6s/step - loss: 1.4438 - accuracy: 0.4664 - val_loss: 0.2463 - val_accuracy: 0.9159
Epoch 2/5
275/275 [==============================] - 1778s 6s/step - loss: 0.2986 - accuracy: 0.9075 - val_loss: 0.1087 - val_accuracy: 0.9655
Epoch 3/5
275/275 [==============================] - 1777s 6s/step - loss: 0.1379 - accuracy: 0.9606 - val_loss: 0.0806 - val_accuracy: 0.9764
Epoch 4/5
275/275 [==============================] - 1775s 6s/step - loss: 0.0906 - accuracy: 0.9735 - val_loss: 0.0739 - val_accuracy: 0.9786
Epoch 5/5
275/275 [==============================] - 1767s 6s/step - loss: 0.0777 - accuracy: 0.9806 - val_loss: 0.0439 - val_accuracy: 0.9855


In [36]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Evaluate CNN model
cnn_y_pred = cnn_model.predict(X_val)
cnn_accuracy = accuracy_score(y_val, cnn_y_pred.argmax(axis=1))
cnn_f1 = f1_score(y_val, cnn_y_pred.argmax(axis=1), average='weighted')

# Evaluate ResNet model
resnet_y_pred = resnet_model.predict(X_val)
resnet_accuracy = accuracy_score(y_val, resnet_y_pred.argmax(axis=1))
resnet_f1 = f1_score(y_val, resnet_y_pred.argmax(axis=1), average='weighted')

# Evaluate VGG model
vgg_y_pred = vgg_model.predict(X_val_rgb)
vgg_accuracy = accuracy_score(y_val, vgg_y_pred.argmax(axis=1))
vgg_f1 = f1_score(y_val, vgg_y_pred.argmax(axis=1), average='weighted')

# Print the evaluation results
print("CNN Accuracy:", cnn_accuracy)
print("CNN F1 Score:", cnn_f1)
print("ResNet Accuracy:", resnet_accuracy)
print("ResNet F1 Score:", resnet_f1)
print("VGG Accuracy:", vgg_accuracy)
print("VGG F1 Score:", vgg_f1)

# Calculate and print confusion matrix
cnn_conf_matrix = confusion_matrix(y_val, cnn_y_pred.argmax(axis=1))
resnet_conf_matrix = confusion_matrix(y_val, resnet_y_pred.argmax(axis=1))
vgg_conf_matrix = confusion_matrix(y_val, vgg_y_pred.argmax(axis=1))
print("CNN Confusion Matrix:")
print(cnn_conf_matrix)
print("ResNet Confusion Matrix:")
print(resnet_conf_matrix)
print("VGG Confusion Matrix:")
print(vgg_conf_matrix)


69/69 [==============================] - 106s 2s/step
CNN Accuracy: 0.9831818181818182
CNN F1 Score: 0.983127175311417
ResNet Accuracy: 0.6040909090909091
ResNet F1 Score: 0.5982500357019189
VGG Accuracy: 0.9854545454545455
VGG F1 Score: 0.9854192271588115
CNN Confusion Matrix:
[[184   0   0   0   0   0   0   0   0   0   0]
 [  0 213   1   0   0   0   0   0   0   0   0]
 [  0   5 189   3   0   0   0   0   0   0   0]
 [  0   1   3 199   0   1   0   0   0   0   0]
 [  1   0   0   0 206   0   0   0   0   3   0]
 [  0   0   0   1   0 176   0   1   0   0   0]
 [  0   1   0   0   0   0 196   0   0   1   0]
 [  0   0   0   1   1   4   1 181   0   5   1]
 [  0   0   0   0   0   0   0   0 213   0   0]
 [  0   0   0   0   0   0   1   0   0 204   1]
 [  0   0   0   0   0   0   0   0   0   0 202]]
ResNet Confusion Matrix:
[[158   0   0   0   8   2   2   1   1   7   5]
 [  0 150  46   3   2   0   1   9   0   0   3]
 [  0  19 120  14   4  28   0   6   0   2   4]
 [  1   5  44  75   1  56   6   2   0